In [2]:
import numpy as np
import matplotlib.pyplot as plt
import scipy
import pandas as pd

In [3]:
%matplotlib inline

In [4]:
# npz_path = "/data/tables/frenet/constraints_12ey_7delta_11vxcar_11vycar_5vxgoal_11wz_11epsi_3curv_mu1.0999999999999999_cs5.0_sorted.npz"
npz_path = "/data/tables/frenet/constraints_12ey_7delta_11vxcar_11vycar_5vxgoal_11wz_13epsi_3curv_mu1.0999999999999999_cs5.0_wider_angles_sorted.npz"
data = np.load(npz_path)
inputs, outputs, constraints = data["inputs"], data["outputs"], data["constraints"]

In [5]:
inputs.shape

(21801780, 8)

In [6]:
try:
    c_data = np.load(npz_path[:-4] + f"_unique_constraints" + npz_path[-4:])
    constraint_types, unique_indices, unique_inverse, unique_counts = c_data["constraint_types"], c_data["unique_indices"], c_data["unique_inverse"], c_data["unique_counts"]
except:
    print("no save, recalcing")
    constraint_types, unique_indices, unique_inverse, unique_counts = np.unique(constraints, return_index=True, return_counts=True, return_inverse=True, axis=0)
    # remove infeasible
    constraint_types = constraint_types[1:]
    unique_indices = unique_indices[1:]
    unique_counts = unique_counts[1:]
    np.savez_compressed(
        npz_path[:-4] + f"_unique_constraints" + npz_path[-4:],
        constraint_types=constraint_types,
        unique_indices=unique_indices,
        unique_inverse=unique_inverse,
        unique_counts=unique_counts,
    )

no save, recalcing


In [11]:
unique_counts_sorted_ind = np.argsort(unique_counts)[::-1]

In [12]:
constraint_types_count_descending = constraint_types[unique_counts_sorted_ind]
unique_indices_count_descending = unique_indices[unique_counts_sorted_ind]
unique_counts_count_descending = unique_counts[unique_counts_sorted_ind]

In [13]:
topk = 499

In [15]:
# convert to onehot
from tqdm import tqdm
id_int = -1 * np.ones((inputs.shape[0]), dtype=int)
for k in tqdm(range(topk)):
    top_ind = np.where((constraints == constraint_types_count_descending[k]).all(axis=1))
    id_int[top_ind] = k
id_int[np.where(id_int == -1)] = topk

100%|██████████| 499/499 [21:11<00:00,  2.55s/it]


In [30]:
id_int2 = id_int.copy()

In [31]:
id_int2[np.where(id_int >399)] = 399

In [32]:
np.savez_compressed(npz_path[:-4] + f"_399_cluster_ids" + npz_path[-4:], cluster_int_ids=id_int2)

In [17]:
np.savez_compressed(npz_path[:-4] + f"_{topk}_cluster_ids" + npz_path[-4:], cluster_int_ids=id_int)

In [18]:
dim_keys = ["ey", "delta", "vx", "vy", "vx_goal", "wz", "epsi", "curv"]

In [39]:
for k in range(10, 500):
    top_ind = np.where((constraints == constraint_types_count_descending[k]).all(axis=1))
    top_inputs = inputs[top_ind]
    top_outputs = outputs[top_ind]
    print((top_inputs.shape, top_outputs.shape))
    np.savez_compressed(f"/data/tables/frenet/test_top{k}.npz", inputs=top_inputs, outputs=top_outputs)

((155220, 8), (155220, 5, 2))
((149712, 8), (149712, 5, 2))
((148862, 8), (148862, 5, 2))
((140440, 8), (140440, 5, 2))
((139531, 8), (139531, 5, 2))
((138746, 8), (138746, 5, 2))
((138345, 8), (138345, 5, 2))
((137783, 8), (137783, 5, 2))
((131016, 8), (131016, 5, 2))
((125322, 8), (125322, 5, 2))
((123216, 8), (123216, 5, 2))
((122339, 8), (122339, 5, 2))
((115166, 8), (115166, 5, 2))
((114586, 8), (114586, 5, 2))
((111362, 8), (111362, 5, 2))
((109505, 8), (109505, 5, 2))
((104716, 8), (104716, 5, 2))
((104002, 8), (104002, 5, 2))
((101726, 8), (101726, 5, 2))
((99715, 8), (99715, 5, 2))
((97069, 8), (97069, 5, 2))
((93705, 8), (93705, 5, 2))
((90244, 8), (90244, 5, 2))
((87757, 8), (87757, 5, 2))
((86471, 8), (86471, 5, 2))
((85640, 8), (85640, 5, 2))
((85061, 8), (85061, 5, 2))
((79993, 8), (79993, 5, 2))
((79752, 8), (79752, 5, 2))
((78588, 8), (78588, 5, 2))
((78537, 8), (78537, 5, 2))
((78404, 8), (78404, 5, 2))
((77927, 8), (77927, 5, 2))
((72981, 8), (72981, 5, 2))
((70824, 8

In [19]:
all_state_means = []
all_state_modes = []
for i in range(topk):
    topi_ind = np.where((constraints == constraint_types_count_descending[i]).all(axis=1))
    topi_inputs = pd.DataFrame(inputs[topi_ind])
    
    mean = topi_inputs.mean(axis=0).to_numpy()
    all_state_means.append(mean)
    mode = topi_inputs.mode(axis=0).to_numpy()
    all_state_modes.append(mode)
    
    # fig, axs = plt.subplots(nrows=num_dim, ncols=1, figsize=(8, 20))
    # for j in range(num_dim):
    #     axs[j].hist(topi_inputs[:, j])
    #     axs[j].set_xlabel(dim_keys[j])
    # plt.subplots_adjust(hspace=0.5)
    # plt.savefig(f"./constraint_cluster_state_space_plots/{i}th_most_frequent.png")
    # plt.close()
    

In [34]:
all_state_modes[39]

array([[ 0.8,  0. ,  2.8,  0. ,  5. , -2.6, -0.8,  0.1]])

In [21]:
for i, i_mode in enumerate(all_state_modes):
    if np.any(i_mode.shape[0] > 1):
        all_state_modes[i] = all_state_modes[i][0][None, :]
        print(i)

39
114
166
175
186
202
203
260
304
328
330
343
384
385
388
472
522
597
606
630
641
649
683
684
693
709
711
716
759
760
771
807
834
883
902
908
943
968
998
1000
1002
1052
1053
1072
1099
1103
1111
1129
1131
1135
1191
1218
1238
1246
1259
1265
1283
1289
1309
1332
1334
1345
1349
1350
1361
1365
1368
1388
1406
1409
1427
1483
1489
1499
1530
1532
1543
1544
1554
1565
1572
1584
1588
1614
1627
1629
1678
1697
1703
1710
1713
1729
1735
1740
1743
1744
1755
1756
1767
1771
1772
1774
1782
1785
1795
1801
1804
1805
1851
1875
1880
1888
1893
1914
1919
1925
1931
1941
1949
1953
1954
1972
1976
1977
1986
1989
1990
2000
2031
2038
2044
2052
2089
2097
2106
2108
2116
2119
2123
2138
2142
2163
2166
2174
2177
2181
2184
2186
2191
2199
2201
2215
2233
2235
2256
2259
2266
2269
2277
2280
2289
2297
2300
2306
2311
2312
2319
2329
2341
2363
2368
2370
2374
2390
2399
2412
2413
2416
2425
2434
2435
2439
2441
2446
2448
2450
2451
2458
2465
2468
2470
2482
2509
2516
2518
2519
2526
2540
2542
2551
2574
2584
2606
2609
2610
2625
2630
2635


In [22]:
as_mean_np = np.array(all_state_means)


In [23]:
as_mode_np = np.array(all_state_modes)[:, 0, :]

In [24]:
np.savez_compressed(npz_path[:-4] + f"_top{topk}mean" + npz_path[-4:], centers=as_mean_np)

In [25]:
np.savez_compressed(npz_path[:-4] + f"_top{topk}mode" + npz_path[-4:], centers=as_mode_np)

In [26]:
df = pd.DataFrame(topi_inputs)

In [27]:
df.mode(axis=0).to_numpy()

array([[ 1.8, -0.3,  5.8,  1. ,  4. , -2.6,  0.8, -0.1]])

In [28]:
df.mean(axis=0).to_numpy()

array([ 1.8, -0.3,  5.8,  1. ,  4. , -2.6,  0.8, -0.1])

In [29]:
df.apply(lambda x: x.value_counts(), axis=0)

,0,1,2,3,4,5,6,7
-2.6,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN
-0.3,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN
-0.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
0.8,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN
1.0,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN
1.8,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4.0,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN
5.8,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN


In [30]:
as_mode_np.shape

(41124, 8)

In [31]:
np.unique(as_mean_np, axis=0).shape

(41118, 8)